In [29]:
import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)



In [30]:
#ADDING THE DATASET

path =("D:\project\Dataset_1.csv")
data=pd.read_csv(path)


In [31]:
#
#data['source'].value_counts()

In [32]:
lines=data[data['source']=='indic2012']
lines.head(20)

,source,english_sentence,hindi_sentence
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
11,indic2012,Category: Religious Text,श्रेणी:धर्मग्रन्थ
12,indic2012,This period summarily is pepped up with devotion.,यह काल समग्रतः भक्ति भावना से ओतप्रोत काल है।
16,indic2012,"And now at present the naturecure, Ayurvedic and modern treatments are taking place through the government in Nepal.","हाल में नेपाल के हस्पताल सामन्यतया आयुर्वेद, प्राकृतिक चिकित्सा तथा आधुनिक चिकीत्सा करके सरकारी सेवा विद्यमान हे ।"
17,indic2012,Parliament time frame is 5 years and this will be dissolved before that.,लोकसभा की कार्यावधि 5 वर्ष है पर्ंतु इसे समय से पूर्व भंग किया जा सकता है
19,indic2012,"Extreme weather due to increased mortality; displacements and economic loss will be compounded through growing population. Although, temperate climate has some benefits out of it such as decreased mortality due to cold weather.",बढ़ती हुई मौतों displacements और आर्थिक नुकसान जो की अतिवादी मौसम (extreme weather)के कारण संभावित हैं बढती हुई जनसँख्या (growing population)के कारण और भी बदतर हो सकते हैं . हालांकि शीतोष्ण क्षेत्र में इसके कुछ फैदे भी हो सकते हैं जैसे की ठंड की वजह से कम मौतें होना .
22,indic2012,"Islam is the world's second-largest religion, after Christianity.",इस्लाम धर्म (الإسلام) ईसाई धर्म के बाद अनुयाइयों के आधार पर दुनिया का दूसरा सब से बड़ा धर्म है।


In [33]:
#Checking for null values
pd.isnull(lines).sum() 

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [34]:
#Dropping null values if any
lines=lines[~pd.isnull(lines['english_sentence'])]

In [35]:
#Dropping duplicates values if any
lines.drop_duplicates(inplace=True)

In [36]:
#Extracting Small Part of Dataset to cross check if any null values are present

lines=lines.sample(n=25000,random_state=42)
lines
#lines.shape


,source,english_sentence,hindi_sentence
43137,indic2012,Occurring in the environment recently increased carbon di Oxide (CO2) monthly CO2 measurements Dershat that if you saw all year get to see the seasonal changes are small; every year Paryaverton Northern Hemisphere (Northern Hemisphere) seasons in the Aswh At the end of the Northern Hemisphere crops have become more Bjne is the time get at the change of environment because the plants remove some CO2 charge,वातावरण में कार्बन डाइऑक्साइड में हाल ही में होने वाली बढोतरी ( CO2 ) .मासिक CO2 मापन यह दर्शाते हैं कि अगर सारे वर्ष को देखा जाए तो छोटे-छोटे मौसमी परिवर्तन देखने को मिलते हैं ; हर साल यह परिवर्तन उत्तरी गोलार्ध (Northern Hemisphere) में वसव्त मौसम के आख़िर में अधिक हो जाते हैं और जब उत्तरी गोलार्ध में फसलें बीजने का समय होता है तो यह परिवर्तन कम हो जाते हैं क्यूंकि पौधे वातावरण में से कुछ CO2 हटा लेते हैंI
107772,indic2012,Her father Aghoranath Chattopadhyay was a well-known scholar and mother was a poet who wrote in Bengali.,इनके पिता अघोरनाथ चट्टोपाध्याय एक नामी विद्वान तथा माँ कवयित्री थीं और बांग्ला में लिखती थीं ।
120507,indic2012,Nishadraj Guh served those three very well in Ringverpur.,ऋंगवेरपुर में निषादराज गुह ने तीनों की बहुत सेवा की।
28755,indic2012,"2. The ordinance that was rejected by Loksabha, can be passed again before six weeks period.",2. लोकसभा एक अध्यादेश को अस्वीकृत करने वाला प्रस्ताव 6 सप्ताह की अवधि समाप्त होने से पूर्व पास कर सकती है
86303,indic2012,Number of 18,अठारह की संख्या
...,...,...,...
117657,indic2012,"There are 6 political parties in India, But the Indian national congress is the most powerful in India",बहुदलिय राजनीतिक व्यवस्था वाले इस देश में ६ राष्ट्रीय स्तर के राजनीतिक दल हैं किंतु यहां की राजनीतिक व्यवस्था पर सर्वाधीक प्रभावी भारतीय राष्ट्रीय कांग्रेस रही है।
89633,indic2012,They had the experience of ruling.,उन्हें शासन करने का अनुभव है ।
127215,indic2012,Ram Kabir comprehensive nature of the concept and wanted to give a different,कबीर राम की अवधारणा को एक भिन्न और व्यापक स्वरूप देना चाहते थे।
67194,indic2012,"this country has many famous places like Agra, Aligad,Ayodhya,Berle,Me rut,Varanasi, (Banaras)Gorakhpur,Ghaziabad,Muradabad,Saharanpur, Faisalabad,Kanpur,",प्रदेश के अन्य महत्त्वपूर्ण शहर हैं आगरा अलीगढ अयोध्या बरेली मेरठ वाराणसी( बनारस) गोरखपुर गाज़ियाबाद मुरादाबाद सहारनपुर फ़ैज़ाबाद कानपुर।


In [37]:
#USING LAMBDA FUNCTIIONS

In [38]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [39]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [40]:
# Set of all special characters
exclude = set(string.punctuation) 
#print(exclude)

# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))


In [41]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
print(remove_digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

{48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None}


In [42]:
# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())


lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [43]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [44]:
lines.head()

,source,english_sentence,hindi_sentence
43137,indic2012,occurring in the environment recently increased carbon di oxide co monthly co measurements dershat that if you saw all year get to see the seasonal changes are small every year paryaverton northern hemisphere northern hemisphere seasons in the aswh at the end of the northern hemisphere crops have become more bjne is the time get at the change of environment because the plants remove some co charge,START_ वातावरण में कार्बन डाइऑक्साइड में हाल ही में होने वाली बढोतरी co मासिक co मापन यह दर्शाते हैं कि अगर सारे वर्ष को देखा जाए तो छोटेछोटे मौसमी परिवर्तन देखने को मिलते हैं हर साल यह परिवर्तन उत्तरी गोलार्ध northern hemisphere में वसव्त मौसम के आख़िर में अधिक हो जाते हैं और जब उत्तरी गोलार्ध में फसलें बीजने का समय होता है तो यह परिवर्तन कम हो जाते हैं क्यूंकि पौधे वातावरण में से कुछ co हटा लेते हैंi _END
107772,indic2012,her father aghoranath chattopadhyay was a wellknown scholar and mother was a poet who wrote in bengali,START_ इनके पिता अघोरनाथ चट्टोपाध्याय एक नामी विद्वान तथा माँ कवयित्री थीं और बांग्ला में लिखती थीं । _END
120507,indic2012,nishadraj guh served those three very well in ringverpur,START_ ऋंगवेरपुर में निषादराज गुह ने तीनों की बहुत सेवा की। _END
28755,indic2012,the ordinance that was rejected by loksabha can be passed again before six weeks period,START_ लोकसभा एक अध्यादेश को अस्वीकृत करने वाला प्रस्ताव सप्ताह की अवधि समाप्त होने से पूर्व पास कर सकती है _END
86303,indic2012,number of,START_ अठारह की संख्या _END


In [45]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [57]:
len(all_eng_words)

30447

In [58]:
len(all_hindi_words)


20506

In [48]:
#Calculating length of each sentence
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [49]:
lines.head()


,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
43137,indic2012,occurring in the environment recently increased carbon di oxide co monthly co measurements dershat that if you saw all year get to see the seasonal changes are small every year paryaverton northern hemisphere northern hemisphere seasons in the aswh at the end of the northern hemisphere crops have become more bjne is the time get at the change of environment because the plants remove some co charge,START_ वातावरण में कार्बन डाइऑक्साइड में हाल ही में होने वाली बढोतरी co मासिक co मापन यह दर्शाते हैं कि अगर सारे वर्ष को देखा जाए तो छोटेछोटे मौसमी परिवर्तन देखने को मिलते हैं हर साल यह परिवर्तन उत्तरी गोलार्ध northern hemisphere में वसव्त मौसम के आख़िर में अधिक हो जाते हैं और जब उत्तरी गोलार्ध में फसलें बीजने का समय होता है तो यह परिवर्तन कम हो जाते हैं क्यूंकि पौधे वातावरण में से कुछ co हटा लेते हैंi _END,67,81
107772,indic2012,her father aghoranath chattopadhyay was a wellknown scholar and mother was a poet who wrote in bengali,START_ इनके पिता अघोरनाथ चट्टोपाध्याय एक नामी विद्वान तथा माँ कवयित्री थीं और बांग्ला में लिखती थीं । _END,17,19
120507,indic2012,nishadraj guh served those three very well in ringverpur,START_ ऋंगवेरपुर में निषादराज गुह ने तीनों की बहुत सेवा की। _END,9,12
28755,indic2012,the ordinance that was rejected by loksabha can be passed again before six weeks period,START_ लोकसभा एक अध्यादेश को अस्वीकृत करने वाला प्रस्ताव सप्ताह की अवधि समाप्त होने से पूर्व पास कर सकती है _END,15,21
86303,indic2012,number of,START_ अठारह की संख्या _END,2,5


In [50]:
#Lines having length greater than 30.
lines[lines['length_eng_sentence']>30].shape


(2030, 5)

In [59]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [52]:
lines.shape

(16970, 5)

In [53]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [62]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence']) #Target sentence having length 20
print(max_length_src)
print(max_length_tar)

20
20


In [64]:
input_words = sorted(list(all_eng_words))
#print(input_words)
target_words = sorted(list(all_hindi_words))
#print(target_words)
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens #Prints the total number of words


(30447, 20506)

In [73]:
#For zero padding
num_decoder_tokens = num_decoder_tokens + 1

In [107]:
#Converting the list of words into dictionary using enumerate( ) function for automatic indexing.

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
#print(input_token_index)
#print(target_token_index)
print(len(input_token_index.keys()))
print(len(target_token_index))
lines

30447
20506


,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
90504,indic2012,meri kavitai ki aadhi sadi half century of my being poet,START_ मेरी कविताई की आधी सदी _END,11,7
50833,indic2012,ames research center,START_ एम्स अनुसंधान केंद्र _END,3,5
77153,indic2012,then drona became the commanderinchief of the army,START_ तब आचार्य द्रोण ने सेनापतित्व का भार ग्रहण किया। _END,8,11
73502,indic2012,then it was divided into north india secret and persia of sasaani empire,START_ इसके बाद यह उत्तर भारत के गुप्त और फारस के सासानी साम्राज्य के बीच बँटा रहा । _END,13,19
59955,indic2012,terrorism reasons of terrorisma special report,START_ आतंकवाद आतंकवाद विरोधी कारणों और आतंकवाद का उपयोग करता है पर एक विश्लेषण _END,6,15
...,...,...,...,...,...
56563,indic2012,sawai jaisingh ii built sisodia rani ka bagsisodia queen garden for his sisodia queen,START_ सवाई जय सिंह ii ने अपनी सिसोदिया रानी के लिए सिसोदिया रानी का बाग बनवाया। _END,14,17
95036,indic2012,khayam ki madhushala khayams bar,START_ खय्याम की मधुशाला _END,5,5
96771,indic2012,part five chapter parliament,START_ भाग पाँच अध्याय संसद _END,4,6
72424,indic2012,washington dc capital of columbia,START_ वाशिंगटन डीसी डिस्ट्रिक्ट अव कोलंबिया _END,5,7


In [99]:
#Converting the same dataset in reverse order to keep index as numeric and value as the word.

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
#print(reverse_input_char_index)
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
#print(reverse_target_char_index)


In [93]:
lines = shuffle(lines)
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
90504,indic2012,meri kavitai ki aadhi sadi half century of my being poet,START_ मेरी कविताई की आधी सदी _END,11,7
50833,indic2012,ames research center,START_ एम्स अनुसंधान केंद्र _END,3,5
77153,indic2012,then drona became the commanderinchief of the army,START_ तब आचार्य द्रोण ने सेनापतित्व का भार ग्रहण किया। _END,8,11
73502,indic2012,then it was divided into north india secret and persia of sasaani empire,START_ इसके बाद यह उत्तर भारत के गुप्त और फारस के सासानी साम्राज्य के बीच बँटा रहा । _END,13,19
59955,indic2012,terrorism reasons of terrorisma special report,START_ आतंकवाद आतंकवाद विरोधी कारणों और आतंकवाद का उपयोग करता है पर एक विश्लेषण _END,6,15


In [94]:
#SPLITIING THE DATASET IN TRAINING AND TESTING SETS.

In [104]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33,random_state=42) #Using keras to split arrays into random train and test subsets.
X_train.shape, X_test.shape

((11369,), (5601,))

In [106]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [109]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [110]:
latent_dim=300

In [111]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [113]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [114]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [115]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    9134100     ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    6152100     ['input_3[0][0]']                
                                                                                            

In [116]:

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [117]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

C:\Users\kpriy\AppData\Local\Temp/ipykernel_3480/2108290721.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/100
88/88 [==============================] - 138s 1s/step - loss: 3.5518 - val_loss: 3.3327
Epoch 2/100
88/88 [==============================] - 115s 1s/step - loss: 3.1991 - val_loss: 3.2315
Epoch 3/100
88/88 [==============================] - 162s 2s/step - loss: 3.0697 - val_loss: 3.1399
Epoch 4/100
88/88 [==============================] - 521s 6s/step - loss: 2.9453 - val_loss: 3.0531
Epoch 5/100
88/88 [==============================] - 161s 2s/step - loss: 2.8220 - val_loss: 2.9670
Epoch 6/100
88/88 [==============================] - 1555s 18s/step - loss: 2.6897 - val_loss: 2.8755
Epoch 7/100
88/88 [==============================] - 105s 1s/step - loss: 2.5567 - val_loss: 2.7913
Epoch 8/100
88/88 [==============================] - 104s 1s/step - loss: 2.4214 - val_loss: 2.7030
Epoch 9/100
88/88 [==============================] - 106s 1s/step - loss: 2.2954 - val_loss: 2.6228
Epoch 10/100
88/88 [==============================] - 107s 1s/step - loss: 2.1683 - val_loss: 2.55

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])